In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import time
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split



In [ ]:
# Função para treinar e avaliar o modelo com Random Forest
def train_and_evaluate(X, y, description):
    """Treina o Random Forest e exibe os resultados"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"\n=== Resultados {description} ===")
    print(f"Acurácia: {accuracy:.4f}")
    print("Relatório de Classificação:")
    print(report)

In [ ]:
# Função para treinar e avaliar o modelo com XGBoost
def train_and_evaluate_xgboost(X, y, description):
    """Treina o modelo XGBoost e exibe os resultados"""

    # Divisão dos dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Criar e treinar o modelo XGBoost
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,    # Número de árvores (boosting rounds)
        max_depth=6,         # Profundidade máxima das árvores
        learning_rate=0.1,   # Taxa de aprendizado
        subsample=0.8,       # Amostragem de dados para evitar overfitting
        colsample_bytree=0.8,# Amostragem de features para cada árvore
        random_state=42,
        use_label_encoder=False,  # Evita warning de codificação
        eval_metric="logloss"     # Evita warning de avaliação desatualizada
    )

    xgb_model.fit(X_train, y_train)

    # Fazer previsões
    y_pred = xgb_model.predict(X_test)

    # Avaliar o modelo
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    # Exibir os resultados
    print(f"\n=== Resultados {description} (XGBoost) ===")
    print(f"Acurácia: {accuracy:.4f}")
    print("Relatório de Classificação:")
    print(report)

In [ ]:
# Load the data
#Carregar a nova feature

file_path = 'C:/Users/abreu/Desktop/Mestrado/Wednesday-workingHours.pcap_ISCX.csv'  # replace with the actual file path

df_new_feature = pd.read_csv("C:/Users/abreu/Desktop/Mestrado/nova_feature.csv")  # Atualize com o caminho correto

df=pd.read_csv(file_path)

In [ ]:
# 2. Pré-processar os dados
# Remover colunas irrelevantes ou com valores constantes
irrelevant_columns = [col for col in df.columns if df[col].nunique() <= 1]
df_reduced = df.drop(columns=irrelevant_columns)


df_reduced.replace([np.inf, -np.inf], np.nan, inplace=True)


# Converter colunas categóricas para numéricas antes de preencher valores ausentes
for col in df_reduced.select_dtypes(include=['object']).columns:
    df_reduced[col] = df_reduced[col].astype('category').cat.codes

# Substituir valores NaN por mediana de cada coluna
df_reduced = df_reduced.fillna(df_reduced.median())


# Preencher valores ausentes com a mediana
df_reduced = df_reduced.fillna(df_reduced.median())


# Separar features e rótulos
X = df_reduced.drop(columns=[' Label'])  # Substitua ' Label' pelo nome correto da coluna de rótulos
y = df_reduced[' Label']


# Converter rótulos para numéricos
y = y.astype('category').cat.codes

# Selecionar apenas colunas numéricas
X = X.select_dtypes(include=[float, int])


In [ ]:
# Aplicar SelectKBest para escolher as 4 melhores features
selector = SelectKBest(score_func=f_classif, k=4)
X_selected = selector.fit_transform(X, y)

# Obter os nomes das features selecionadas (opcional)
selected_features = X.columns[selector.get_support()]
print("As 4 melhores features selecionadas são:", selected_features.tolist())

As 4 melhores features selecionadas são: ['Bwd Packet Length Max', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Avg Bwd Segment Size']


In [ ]:
# Pegar os scores apenas das 4 features selecionadas
selected_scores = selector.scores_[selector.get_support()]

# Juntar nomes e scores em um DataFrame
selected_features_df = pd.DataFrame({
    'Feature': selected_features,
    'Score': selected_scores
}).sort_values(by='Score', ascending=False).reset_index(drop=True)

print("Scores das 4 features selecionadas:")
print(selected_features_df)

Scores das 4 features selecionadas:
                   Feature          Score
0   Bwd Packet Length Mean  112023.055569
1     Avg Bwd Segment Size  112023.055569
2    Bwd Packet Length Std  108310.460441
3    Bwd Packet Length Max  106125.975216


In [ ]:
# Verificar número de linhas da matriz de features selecionadas pelo SelectKBest
num_rows_selected_features = X_selected.shape[0]
print(num_rows_selected_features)

692703


In [ ]:
# Criar versão dos dados com a nova feature incluída
X_with_new_feature = np.column_stack((X_selected, df_new_feature.values))

In [ ]:
train_and_evaluate(X_selected, y, "SEM a nova feature")
train_and_evaluate(X_with_new_feature, y, "COM a nova feature")


=== Resultados SEM a nova feature ===
Acurácia: 0.8744
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91    132009
           1       1.00      0.73      0.84      3088
           2       1.00      0.68      0.81     69322
           3       0.99      0.07      0.13      1650
           4       0.82      0.29      0.43      1739
           5       1.00      1.00      1.00         3

    accuracy                           0.87    207811
   macro avg       0.94      0.63      0.69    207811
weighted avg       0.89      0.87      0.86    207811


=== Resultados COM a nova feature ===
Acurácia: 0.9562
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97    132009
           1       1.00      0.72      0.83      3088
           2       0.93      0.96      0.95     69322
           3       1.00      0.06      0.12      1650
           4      

In [ ]:
# Testar o modelo SEM a nova feature
train_and_evaluate_xgboost(X_selected, y, "SEM a nova feature")

# Testar o modelo COM a nova feature
train_and_evaluate_xgboost(X_with_new_feature, y, "COM a nova feature")


C:\Users\abreu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [17:38:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



=== Resultados SEM a nova feature (XGBoost) ===
Acurácia: 0.8730
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91    132009
           1       0.99      0.68      0.81      3088
           2       1.00      0.68      0.81     69322
           3       0.91      0.07      0.14      1650
           4       0.68      0.29      0.41      1739
           5       1.00      1.00      1.00         3

    accuracy                           0.87    207811
   macro avg       0.90      0.62      0.68    207811
weighted avg       0.89      0.87      0.86    207811



C:\Users\abreu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [17:38:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



=== Resultados COM a nova feature (XGBoost) ===
Acurácia: 0.9548
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97    132009
           1       0.99      0.72      0.83      3088
           2       0.93      0.96      0.95     69322
           3       0.89      0.08      0.15      1650
           4       0.76      0.26      0.39      1739
           5       1.00      1.00      1.00         3

    accuracy                           0.95    207811
   macro avg       0.92      0.67      0.71    207811
weighted avg       0.95      0.95      0.95    207811

